In [1163]:
# -*- coding: utf-8 -*
#!/usr/bin/env python3

import json
import datetime as dt
import urllib.request
import pandas as pd
import numpy as np

from sqlalchemy import Column, Integer, Float, String
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy import inspect

requestURL = "https://eodhistoricaldata.com/api/eod/"
myEodKey = "5ba84ea974ab42.45160048"

startDate = dt.datetime(2018,1,1)
endDate = dt.datetime(2019,1,31)

backtest_start = dt.datetime(2018,12,31)
backtest_end = dt.datetime(2019,1,31)


def get_daily_data(symbol, start=startDate, end=endDate, requestType=requestURL, apiKey=myEodKey):
    symbolURL = str(symbol) + ".US?"
    startURL = "from=" + str(start)
    endURL = "to=" + str(end)
    apiKeyURL = "api_token=" + myEodKey
    completeURL = requestURL + symbolURL + startURL + '&' + endURL + '&' + apiKeyURL + '&period=d&fmt=json'
#     print(completeURL)
    with urllib.request.urlopen(completeURL) as req:
        data = json.load(req)
        return data
    
def create_pair_table(name, metadata, engine):
	tables = metadata.tables.keys()
	if name not in tables:
		table = Table(name, metadata, 
					Column('symbol', String(50), primary_key=True, nullable=False),
					Column('date', String(50), primary_key=True, nullable=False),
					Column('open', Float, nullable=False),
					Column('high', Float, nullable=False),
					Column('low', Float, nullable=False),
					Column('close', Float, nullable=False),
                       Column('adjusted_close', Float, nullable=False),
					Column('volume', Integer, nullable=False))
		table.create(engine)

def clear_a_table(table_name, metadata, engine):
    conn = engine.connect()
    table = metadata.tables[table_name]
    delete_st = table.delete()
    conn.execute(delete_st)

# def populate_stock_data(tickers, metadata, engine, table_name):
#     conn = engine.connect()
#     table = metadata.tables[table_name]
#     for ticker in tickers:
#         stock = get_daily_data(ticker)
# #         print(stock)
#         for stock_data in stock:
#             #print(k, v)
#             trading_date = stock_data['date']
#             trading_open = stock_data['open']
#             trading_high = stock_data['high']
#             trading_low = stock_data['low']
#             trading_close = stock_data['close']
#             trading_adjusted_close = stock_data['adjusted_close']
#             trading_volume = stock_data['volume']
#             insert_st = table.insert().values(symbol=ticker, date=trading_date,
# 					open = trading_open, high = trading_high, low = trading_low,
# 					close = trading_close, adjusted_close = trading_adjusted_close, 
#                        volume = trading_volume)
#             conn.execute(insert_st)

def execute_sql_statement(sql_st, engine):
    result = engine.execute(sql_st)
    return result

In [1164]:
def populate_stock_data(ticker, metadata, engine, table_name):
    conn = engine.connect()
    table = metadata.tables[table_name]

    stock = get_daily_data(ticker)
    for stock_data in stock:
        trading_date = stock_data['date']
        trading_open = stock_data['open']
        trading_high = stock_data['high']
        trading_low = stock_data['low']
        trading_close = stock_data['close']
        trading_adjusted_close = stock_data['adjusted_close']
        trading_volume = stock_data['volume']
        insert_st = table.insert().values(symbol=ticker, date=trading_date,
				open = trading_open, high = trading_high, low = trading_low,
				close = trading_close, adjusted_close = trading_adjusted_close, 
                   volume = trading_volume)
        conn.execute(insert_st)

In [1165]:
def build_pair_trading_model():
    # ............
	# ............
  
    return 0

In [1166]:
def create_pairs_table(name, metadata, engine):
    tables = metadata.tables.keys()
    if name not in tables:
        table = Table(name, metadata, 
                        Column('ticker1', String(50), primary_key=True, nullable=False),
                        Column('ticker2', String(50), primary_key=True, nullable=False),
                        Column('volatility', Float, nullable=False),
                        Column('profit_loss', Float, nullable=False))

        table.create(engine)

In [1167]:
def create_pairprices_table(name, metadata, engine):
    tables = metadata.tables.keys()
    if name not in tables:
        table = Table(name, metadata, 
					Column('ticker1', String(50), primary_key=True, nullable=False),
					Column('ticker2', String(50), primary_key=True, nullable=False),
					Column('date', String(50), primary_key=True, nullable=False),
					Column('open1', Float, nullable=False),
					Column('close1', Float, nullable=False),
					Column('open2', Float, nullable=False),
					Column('close2', Float, nullable=False))
        
        table.create(engine)

In [1168]:
def create_trades_table(name, metadata, engine):
    tables = metadata.tables.keys()
    if name not in tables:
        table = Table(name, metadata, 
					Column('ticker1', String(50), primary_key=True, nullable=False),
					Column('ticker2', String(50), primary_key=True, nullable=False),
					Column('date', String(50), primary_key=True, nullable=False),
					Column('profit_loss', Float, nullable=False))
        
        table.create(engine)

In [1169]:
engine = create_engine('sqlite:///:memory:')
metadata = MetaData(engine)

In [1170]:
create_pair_table('Pair1Stocks', metadata, engine)

In [1171]:
create_pair_table('Pair2Stocks', metadata, engine)

In [1172]:
create_pairs_table('Pairs', metadata, engine)

In [1173]:
create_pairprices_table('PairPrices', metadata, engine)

In [1174]:
create_trades_table('Trades', metadata, engine)

In [1175]:
metadata.tables.keys()

dict_keys(['Pair1Stocks', 'Pair2Stocks', 'Pairs', 'PairPrices', 'Trades'])

In [1176]:
tickers = ['AAPL', 'HPQ']

In [1177]:
populate_stock_data(tickers[0], metadata, engine, 'Pair1Stocks')

In [1178]:
populate_stock_data(tickers[1], metadata, engine, 'Pair2Stocks')

In [1179]:
sql_st = 'SELECT * FROM Pair1Stocks'
result = execute_sql_statement(sql_st, engine)

In [1180]:
k = 1

In [1181]:
sql_st = 'SELECT adjusted_close FROM Pair1Stocks'
result1 = execute_sql_statement(sql_st, engine)
sql_st = 'SELECT adjusted_close FROM Pair2Stocks'
result2 = execute_sql_statement(sql_st, engine)

In [1182]:
p1 = np.array(result1.fetchall())
p2 = np.array(result2.fetchall())

In [1183]:
vol = np.std(p1/p2)
vol

0.599085419217168

In [1184]:
sql_st = 'SELECT symbol, date, open, close FROM Pair1Stocks'

In [1185]:
Prices1 = pd.read_sql(sql_st, engine)

In [1186]:
sql_st = 'SELECT symbol, date, open, close FROM Pair2Stocks'

In [1187]:
Prices2 = pd.read_sql(sql_st, engine)

In [1188]:
Prices1.shape

(272, 4)

In [1189]:
Prices2.head()

,symbol,date,open,close
0,HPQ,2018-01-02,21.18,21.32
1,HPQ,2018-01-03,21.31,21.37
2,HPQ,2018-01-04,21.40,21.53
3,HPQ,2018-01-05,21.61,21.75
4,HPQ,2018-01-08,21.78,21.84


In [1190]:
def populate_pairprices(tickers, metadata, engine):
    conn = engine.connect()
    table = metadata.tables['PairPrices']
    
    s = backtest_start.strftime('%Y-%m-%d')
    e = backtest_end.strftime('%Y-%m-%d')
    
    sql_st = '''
SELECT Pair1Stocks.symbol, Pair2Stocks.symbol, Pair1Stocks.date, Pair1Stocks.open, Pair1Stocks.close, Pair2Stocks.open, Pair2Stocks.close 
FROM Pair1Stocks, Pair2Stocks 
WHERE ((Pair1Stocks.date >= \'''' + s + "\' AND Pair1Stocks.date <= \'" + e + "\') AND (Pair1Stocks.date = Pair2Stocks.date))"
    
    result = execute_sql_statement(sql_st, engine)
    
    for r in result:
        insert_st = table.insert().values(ticker1=tickers[0], ticker2=tickers[1], date=r[2],
					open1 = r[3], close1 = r[4],
					open2 = r[5], close2 = r[6])
    
        conn.execute(insert_st)

In [1191]:
tickers = ['AAPL', 'HPQ']

In [1192]:
populate_pairprices(tickers, metadata, engine)

In [1193]:
sql_st = 'SELECT * FROM PairPrices'
result = execute_sql_statement(sql_st, engine)

In [1194]:
PairPrices = pd.DataFrame(result.fetchall(), columns=['ticker1', 'ticker2', 'date', 'open1', 'close1', 'open2', 'close2'])

In [1195]:
PairPrices.head()

,ticker1,ticker2,date,open1,close1,open2,close2
0,AAPL,HPQ,2018-12-31,158.53,157.74,20.49,20.46
1,AAPL,HPQ,2019-01-02,154.89,157.92,20.03,20.63
2,AAPL,HPQ,2019-01-03,143.98,142.19,20.35,19.93
3,AAPL,HPQ,2019-01-04,144.53,148.26,20.38,20.72
4,AAPL,HPQ,2019-01-07,148.70,147.93,20.75,20.95


In [1196]:
PnL = []

for i in range(1, len(PairPrices)):
    
    diff = abs( PairPrices.loc[i-1, 'close1'] / PairPrices.loc[i-1, 'close2'] \
    - PairPrices.loc[i, 'open1'] / PairPrices.loc[i, 'open2'] )
    
    if diff >= k*vol:
        # short the pair
        N1 = 10000
        N2 = (-N1) * ( PairPrices.loc[i, 'open1'] / PairPrices.loc[i, 'open2'] )
        # close the trade and calculate PnL
        r = N1 * ( PairPrices.loc[i, 'open1'] - PairPrices.loc[i, 'close1'] ) \
            + N2 * ( PairPrices.loc[i, 'open2'] - PairPrices.loc[i, 'close2'] )
        
    elif diff < k*vol:
        # long the pair
        N1 = -10000
        N2 = (-N1) * ( PairPrices.loc[i, 'open1'] / PairPrices.loc[i, 'open2'] )
        # close the trade and calculate PnL
        r = N1 * ( PairPrices.loc[i, 'open1'] - PairPrices.loc[i, 'close1'] ) \
            + N2 * ( PairPrices.loc[i, 'open2'] - PairPrices.loc[i, 'close2'] )
    
    PnL.append(r)

In [1197]:
PnL.insert(0,0)

In [1198]:
PairPrices['PnL'] = PnL

In [1199]:
PairPrices.head()

,ticker1,ticker2,date,open1,close1,open2,close2,PnL
0,AAPL,HPQ,2018-12-31,158.53,157.74,20.49,20.46,0.000000
1,AAPL,HPQ,2019-01-02,154.89,157.92,20.03,20.63,-16097.403894
2,AAPL,HPQ,2019-01-03,143.98,142.19,20.35,19.93,11815.773956
3,AAPL,HPQ,2019-01-04,144.53,148.26,20.38,20.72,13188.027478
4,AAPL,HPQ,2019-01-07,148.70,147.93,20.75,20.95,-22032.530120


In [1200]:
def populate_trades(tickers, metadata, engine):
    conn = engine.connect()
    table = metadata.tables['Trades']
    
    for i in range(len(PairPrices)):
        
        trading_date = PairPrices.loc[i, 'date']
        pnl = PairPrices.loc[i, 'PnL']
        
        insert_st = table.insert().values(ticker1=tickers[0], ticker2=tickers[1], date=trading_date,
					profit_loss = pnl)
    
        conn.execute(insert_st)

In [1201]:
populate_trades(tickers, metadata, engine)

In [1202]:
sql_st = 'SELECT * FROM Trades'
result = execute_sql_statement(sql_st, engine)

In [1203]:
Trades = pd.DataFrame(result.fetchall(), columns=['ticker1', 'ticker2', 'date', 'PnL'])
Trades.head()

,ticker1,ticker2,date,PnL
0,AAPL,HPQ,2018-12-31,0.000000
1,AAPL,HPQ,2019-01-02,-16097.403894
2,AAPL,HPQ,2019-01-03,11815.773956
3,AAPL,HPQ,2019-01-04,13188.027478
4,AAPL,HPQ,2019-01-07,-22032.530120


In [1204]:
total_PnL = Trades['PnL'].sum()
total_PnL

-6066.149696488279

In [1205]:
def populate_pairs(tickers, metadata, engine):
    conn = engine.connect()
    table = metadata.tables['Pairs']
                    
    insert_st = table.insert().values(ticker1=tickers[0], ticker2=tickers[1], volatility=vol,
				profit_loss = total_PnL)
    
    conn.execute(insert_st)

In [1206]:
populate_pairs(tickers, metadata, engine)

In [1207]:
sql_st = 'SELECT * FROM Pairs'
result = execute_sql_statement(sql_st, engine)

In [1208]:
result.fetchall()

[('AAPL', 'HPQ', 0.599085419217168, -6066.149696488279)]